### **HuggingFace Login**

In [1]:
from huggingface_hub import login
login()

### **Import Libairies**

In [2]:
from IPython.display import display

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from transformers import Trainer, TrainingArguments
import sentencepiece
from transformers import AlbertTokenizer, AlbertForSequenceClassification
from datasets import Dataset



### **Read Datasets**

In [3]:
'''
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv", index_col=0)
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv", index_col=0)

df_train = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/train.csv", index_col=0)
df_test = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/test.csv", index_col=0)
'''

df_train = pd.read_csv("C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/train.csv", index_col=0)
df_test = pd.read_csv("C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/test.csv", index_col=0)


### **Short Analysis**

In [4]:
print(f"Length - train {len(df_train)} - test {len(df_test)}")

Length - train 7613 - test 3263


### **Preprocess Datasets**

##### _Merge columns (full)_

In [5]:
keywords = pd.concat([
    pd.DataFrame(df_train["keyword"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["keyword"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
keywords.head()

,train,test
keyword,,
fatalities,45,5
deluge,42,8
armageddon,42,8
damage,41,9
body%20bags,41,9


In [6]:
locations = pd.concat([
    pd.DataFrame(df_train["location"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["location"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
locations.head()

,train,test
location,,
USA,104.0,37.0
New York,71.0,38.0
United States,50.0,15.0
London,45.0,13.0
Canada,29.0,13.0


In [7]:
df_train_full = df_train.copy()
df_train_full['text'] = df_train_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)
df_test_full = df_test.copy()
df_test_full['text'] = df_test_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)


### **Model Assessment**

In [8]:
def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    metrics_df = pd.DataFrame({
        'Value': [f1, precision, recall]
    }, index=['F1 Score', 'Precision', 'Recall'])
    
    cm_df = pd.DataFrame(cm, columns=['Predicted Negative', 'Predicted Positive'], index=['Actual Negative', 'Actual Positive'])
    
    return metrics_df, cm_df

### **ALBERT Tokenizer**

In [9]:
albert_tokenizer_full = AlbertTokenizer.from_pretrained("albert-base-v2")

albert_encodings_full = albert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
albert_labels_full = torch.tensor(list(df_train_full['target']))

albert_input_ids_train_full, albert_input_ids_valid_full, albert_token_type_ids_train_full, albert_token_type_ids_valid_full, \
albert_attention_mask_train_full, albert_attention_mask_valid_full, albert_y_train_full, albert_y_valid_full = train_test_split(
    albert_encodings_full['input_ids'], 
    albert_encodings_full['token_type_ids'], 
    albert_encodings_full['attention_mask'], 
    albert_labels_full, 
    test_size=0.15, 
    stratify=albert_labels_full, 
    random_state=0
)

albert_train_encodings_full = {
    'input_ids': torch.tensor(albert_input_ids_train_full),
    'token_type_ids': torch.tensor(albert_token_type_ids_train_full),
    'attention_mask': torch.tensor(albert_attention_mask_train_full)
}

albert_valid_encodings_full = {
    'input_ids': torch.tensor(albert_input_ids_valid_full),
    'token_type_ids': torch.tensor(albert_token_type_ids_valid_full),
    'attention_mask': torch.tensor(albert_attention_mask_valid_full)
}

albert_train_dataset_full = Dataset.from_dict({
    "input_ids": albert_train_encodings_full['input_ids'],
    "attention_mask": albert_train_encodings_full['attention_mask'],
    "labels": albert_y_train_full
})

albert_valid_dataset_full = Dataset.from_dict({
    "input_ids": albert_valid_encodings_full['input_ids'],
    "attention_mask": albert_valid_encodings_full['attention_mask'],
    "labels": albert_y_valid_full
})

albert_test_encodings_full = albert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)

albert_test_encodings_full = {
    key: torch.tensor(val) for key, val in albert_test_encodings_full.items()
}

albert_test_dataset_full = Dataset.from_dict({
    "input_ids": albert_test_encodings_full['input_ids'],
    "attention_mask": albert_test_encodings_full['attention_mask']
})

##### _Save HuggingFace ALBERT Tokenizer_

In [12]:
albert_tokenizer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_albert_tokenizer")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\Users\cauchepy\Git\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cauchepy\.cache\huggingface\hub\models--yanncauchepin--kaggle_disastertweets_albert_tokenizer. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/yanncauchepin/kaggle_disastertweets_albert_tokenizer/commit/7a7baa1173f3e18d03a3d765cd13fd98e9ee46d0', commit_message='Upload tokenizer', commit_description='', oid='7a7baa1173f3e18d03a3d765cd13fd98e9ee46d0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yanncauchepin/kaggle_disastertweets_albert_tokenizer', endpoint='https://huggingface.co', repo_type='model', repo_id='yanncauchepin/kaggle_disastertweets_albert_tokenizer'), pr_revision=None, pr_num=None)

### **ALBERT Training from scratch**

##### _ALBERT Transformers + local save_

In [21]:
albert_model_full = AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)

training_args = TrainingArguments(
    hub_model_id="kaggle_disastertweets_albert_model",
    output_dir="./albert_results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./albert_logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

albert_trainer_full = Trainer(
    model=albert_model_full,
    args=training_args,
    train_dataset=albert_train_dataset_full,
    eval_dataset=albert_valid_dataset_full
)

albert_trainer_full.train()

albert_trainer_full.save_model("albert_model") # Local

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/8090 [00:00<?, ?it/s]

{'loss': 0.7248, 'grad_norm': 37.749755859375, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.6746, 'grad_norm': 20.725921630859375, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 0.7108, 'grad_norm': 28.134456634521484, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 0.6825, 'grad_norm': 26.18404197692871, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 0.6595, 'grad_norm': 31.306425094604492, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 0.6241, 'grad_norm': 18.379308700561523, 'learning_rate': 6e-06, 'epoch': 0.04}
{'loss': 0.6434, 'grad_norm': 52.83509063720703, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.04}
{'loss': 0.6161, 'grad_norm': 70.47216033935547, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.6151, 'grad_norm': 62.251914978027344, 'learning_rate': 9e-06, 'epoch': 0.06}
{'loss': 0.6969, 'grad_norm': 68.26067352294922, 'learning_rate': 1e-05, 'epoch': 0.06}
{'loss': 0.5343, 'grad_norm': 46.98

  0%|          | 0/286 [00:00<?, ?it/s]

{'eval_loss': 0.6836364269256592, 'eval_runtime': 9.5865, 'eval_samples_per_second': 119.126, 'eval_steps_per_second': 29.834, 'epoch': 1.0}
{'loss': 0.6633, 'grad_norm': 3.2439804077148438, 'learning_rate': 4.262187088274045e-05, 'epoch': 1.0}
{'loss': 0.7245, 'grad_norm': 6.807226181030273, 'learning_rate': 4.255599472990778e-05, 'epoch': 1.01}
{'loss': 0.6966, 'grad_norm': 17.45539665222168, 'learning_rate': 4.24901185770751e-05, 'epoch': 1.01}
{'loss': 0.6872, 'grad_norm': 6.992506504058838, 'learning_rate': 4.242424242424243e-05, 'epoch': 1.02}
{'loss': 0.6255, 'grad_norm': 6.1654133796691895, 'learning_rate': 4.235836627140975e-05, 'epoch': 1.03}
{'loss': 0.6885, 'grad_norm': 14.984956741333008, 'learning_rate': 4.229249011857708e-05, 'epoch': 1.03}
{'loss': 0.7417, 'grad_norm': 13.459877967834473, 'learning_rate': 4.222661396574441e-05, 'epoch': 1.04}
{'loss': 0.7111, 'grad_norm': 11.418614387512207, 'learning_rate': 4.216073781291173e-05, 'epoch': 1.04}
{'loss': 0.709, 'grad_no

...

{'eval_loss': 0.6997802257537842, 'eval_runtime': 8.3575, 'eval_samples_per_second': 136.644, 'eval_steps_per_second': 34.221, 'epoch': 3.0}
{'loss': 0.7337, 'grad_norm': 3.712514638900757, 'learning_rate': 2.127799736495389e-05, 'epoch': 3.0}
{'loss': 0.6994, 'grad_norm': 3.2234532833099365, 'learning_rate': 2.1212121212121215e-05, 'epoch': 3.01}
{'loss': 0.7054, 'grad_norm': 15.66004467010498, 'learning_rate': 2.114624505928854e-05, 'epoch': 3.02}
{'loss': 0.673, 'grad_norm': 8.311396598815918, 'learning_rate': 2.1080368906455864e-05, 'epoch': 3.02}
{'loss': 0.6543, 'grad_norm': 7.457825183868408, 'learning_rate': 2.101449275362319e-05, 'epoch': 3.03}
{'loss': 0.6975, 'grad_norm': 7.429793357849121, 'learning_rate': 2.0948616600790517e-05, 'epoch': 3.03}
{'loss': 0.6345, 'grad_norm': 5.6346049308776855, 'learning_rate': 2.088274044795784e-05, 'epoch': 3.04}
{'loss': 0.707, 'grad_norm': 5.597073078155518, 'learning_rate': 2.0816864295125166e-05, 'epoch': 3.05}
{'loss': 0.6776, 'grad_n

  0%|          | 0/286 [00:00<?, ?it/s]

{'eval_loss': 0.6847966909408569, 'eval_runtime': 6.8188, 'eval_samples_per_second': 167.478, 'eval_steps_per_second': 41.943, 'epoch': 4.0}
{'loss': 0.6914, 'grad_norm': 9.361568450927734, 'learning_rate': 1.0606060606060607e-05, 'epoch': 4.0}
{'loss': 0.6971, 'grad_norm': 12.667191505432129, 'learning_rate': 1.0540184453227932e-05, 'epoch': 4.01}
{'loss': 0.7268, 'grad_norm': 12.163480758666992, 'learning_rate': 1.0474308300395258e-05, 'epoch': 4.02}
{'loss': 0.7105, 'grad_norm': 12.146296501159668, 'learning_rate': 1.0408432147562583e-05, 'epoch': 4.02}
{'loss': 0.682, 'grad_norm': 10.505599975585938, 'learning_rate': 1.0342555994729908e-05, 'epoch': 4.03}
{'loss': 0.6934, 'grad_norm': 5.090905666351318, 'learning_rate': 1.0276679841897234e-05, 'epoch': 4.04}
{'loss': 0.6828, 'grad_norm': 6.112482070922852, 'learning_rate': 1.0210803689064559e-05, 'epoch': 4.04}
{'loss': 0.6827, 'grad_norm': 8.083372116088867, 'learning_rate': 1.0144927536231885e-05, 'epoch': 4.05}
{'loss': 0.6795, 

  0%|          | 0/286 [00:00<?, ?it/s]

{'eval_loss': 0.6838981509208679, 'eval_runtime': 6.7762, 'eval_samples_per_second': 168.532, 'eval_steps_per_second': 42.207, 'epoch': 5.0}
{'train_runtime': 822.6391, 'train_samples_per_second': 39.331, 'train_steps_per_second': 9.834, 'train_loss': 0.6893342249473035, 'epoch': 5.0}


##### _Save HugggingFace trained ALBERT Transformers_ 

In [27]:
albert_trainer_full.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/yanncauchepin/kaggle_disastertweets_albert_model/commit/bfe302b5985dde92d1a2415e4867588a2df8f0be', commit_message='End of training', commit_description='', oid='bfe302b5985dde92d1a2415e4867588a2df8f0be', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yanncauchepin/kaggle_disastertweets_albert_model', endpoint='https://huggingface.co', repo_type='model', repo_id='yanncauchepin/kaggle_disastertweets_albert_model'), pr_revision=None, pr_num=None)

### **ALBERT Loading from pretrained**

In [19]:
albert_model_full = AlbertForSequenceClassification.from_pretrained("albert_model")

training_args = TrainingArguments(
    hub_model_id="kaggle_disastertweets_albert_model",
    output_dir="./albert_results",
    num_train_epochs=15,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./albert_logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

albert_trainer_full = Trainer(
    model=albert_model_full,
    args=training_args,
    train_dataset=albert_train_dataset_full,
    eval_dataset=albert_valid_dataset_full
)

### **ALBERT Assessment**

In [26]:
albert_predictions_full = albert_trainer_full.predict(albert_valid_dataset_full)
albert_logits_full = albert_predictions_full.predictions
albert_y_pred_full = np.argmax(albert_logits_full, axis=1)

albert_trainer_full_assessement = evaluate_classifier(albert_y_valid_full.numpy(), albert_y_pred_full)
display(albert_trainer_full_assessement[0])
display(albert_trainer_full_assessement[1])

  0%|          | 0/286 [00:00<?, ?it/s]

c:\Users\cauchepy\Git\.venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Value
F1 Score,0.413948
Precision,0.324960
Recall,0.570053


,Predicted Negative,Predicted Positive
Actual Negative,651,0
Actual Positive,491,0


### **ALBERT Test Predictions**

##### _ALBERT Test Predictions_

In [28]:
albert_test_predictions_full = albert_trainer_full.predict(albert_test_dataset_full)
albert_test_logits_full = albert_test_predictions_full.predictions
albert_test_y_pred_full = np.argmax(albert_test_logits_full, axis=1)

albert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': albert_test_y_pred_full.flatten()
})

  0%|          | 0/816 [00:00<?, ?it/s]

##### _Save HuggingFace ALBERT Test Predictions_

In [29]:
hf_albert_test_submission_full = Dataset.from_pandas(albert_test_submission_full)
hf_albert_test_submission_full.push_to_hub("yanncauchepin/kaggle_disastertweets_albert_submission_df")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/299 [00:00<?, ?B/s]

c:\Users\cauchepy\Git\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cauchepy\.cache\huggingface\hub\datasets--yanncauchepin--kaggle_disastertweets_albert_submission_df. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
No files have been modified since last commit. Skipping to prevent empty commit

CommitInfo(commit_url='https://huggingface.co/datasets/yanncauchepin/kaggle_disastertweets_albert_submission_df/commit/e6329f9ba75ac8758b419b75a57ede20e267465b', commit_message='Upload dataset', commit_description='', oid='e6329f9ba75ac8758b419b75a57ede20e267465b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yanncauchepin/kaggle_disastertweets_albert_submission_df', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yanncauchepin/kaggle_disastertweets_albert_submission_df'), pr_revision=None, pr_num=None)